In [ ]:
import os
%pip install pytube
from pytube import YouTube




url = input("Enter the YouTube video URL: ")


output_folder = "pytube_audio_files"
os.makedirs(output_folder, exist_ok=True)


yt = YouTube(url)
audio_stream = yt.streams.filter(only_audio=True).first()
audio_stream.download(output_path=output_folder)


default_filename = audio_stream.default_filename
new_filename = f"{output_folder}/{yt.title}.mp3"
os.rename(f"{output_folder}/{default_filename}", new_filename)
print(f'Download successful! Saved to {new_filename}')


Enter the YouTube video URL: https://www.youtube.com/watch?v=oWKgpB2zpgw
Download successful! Saved to pytube_audio_files/Abhi Mujh Mein Kahin Best Lyric|Agneepath|Priyanka Chopra,Hri....mp3


In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-a30m0ler/demucs_0dbce8df089246068147cca69d3c77b4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-a30m0ler/demucs_0dbce8df089246068147cca69d3c77b4
  Resolved https://github.com/facebookresearch/demucs to commit 0fa7d4511b0fca3f96f59fa37f7dcaed555990d6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.6 MB/s e

In [ ]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/pytube_audio_files'
out_path = '/content/output_after_audio_separation'


In [ ]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [ ]:
separate()

Going to separate the files:
/content/pytube_audio_files/Abhi Mujh Mein Kahin Best Lyric|Agneepath|Priyanka Chopra,Hri....mp3
With command:  python3 -m demucs.separate -o /content/output_after_audio_separation -n htdemucs --mp3 --mp3-bitrate=320


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 155MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/output_after_audio_separation/htdemucs
Separating track /content/pytube_audio_files/Abhi Mujh Mein Kahin Best Lyric|Agneepath|Priyanka Chopra,Hri....mp3


100%|████████████████████████████████████████████████████████████████████████| 257.4/257.4 [09:24<00:00,  2.19s/seconds]


In [ ]:
import os
import shutil

def copy_directory_with_vocals(directory_path, target_directory):
    for root, dirs, files in os.walk(directory_path):
        if 'vocals.mp3' in files:
            source_directory = os.path.dirname(os.path.join(root, 'vocals.mp3'))

            # Copy the directory to the specified target directory
            shutil.copytree(source_directory, os.path.join(target_directory, os.path.basename(source_directory)))
            print(f'Directory containing vocals.mp3 copied to {target_directory}.')


starting_directory = '/content/output_after_audio_separation/htdemucs'


target_directory = '/content/'


copy_directory_with_vocals(starting_directory, target_directory)


Directory containing vocals.mp3 copied to /content/.


In [ ]:
pip install elevenlabs

In [ ]:
import requests

# Set your API key
api_key = "1ddd773d53b066c3d550a3a6fcf564b7"

# Define the URL for the API endpoint
url = "https://api.elevenlabs.io/v1/voices/add"

# Define the headers (including the API key)
headers = {
    "xi-api-key": api_key
}

# Ask the user for input
name = input("Enter the name of the speaker: ")
description = input("Enter a description for the speaker: ")

# Define the payload (request body)
payload = {
    "name": name,
    "description": description
}

# Define the files (audio samples)
files = []

# Assuming you want to add multiple audio samples
while True:
    file_path = input("Enter the path of an audio sample (or press Enter to finish): ")
    if not file_path:
        break
    files.append(("files", open(file_path, "rb")))

# Make the POST request
response = requests.post(url, headers=headers, data=payload, files=files)

# Check the response
if response.status_code == 200:
    print("Voice added successfully.")
else:
    print(f"Error adding voice. Status code: {response.status_code}")
    print(response.text)


Enter the name of the speaker: Sonu Nigam
Enter a description for the speaker: Sonu_nigam_vaibhav
Enter the path of an audio sample (or press Enter to finish): /content/Abhi Mujh Mein Kahin Best Lyric|Agneepath|Priyanka Chopra,Hri.../vocals.mp3
Enter the path of an audio sample (or press Enter to finish): 
Voice added successfully.
